In [21]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import torch
import random
import os
from torchvision import datasets
import torch.utils.data as data
from sklearn.preprocessing import Normalizer
from sklearn.metrics import f1_score
from PIL import Image
from rdkit import Chem
from rdkit.Chem import AllChem

df_data = pd.read_csv('/egr/research-aidd/menghan1/AnchorDrug/data/level5_beta_trt_cp_24h_10uM.csv')
use_HQ_sample_id = pd.read_csv(f'/egr/research-aidd/menghan1/AnchorDrug/HQ_LINCS_retrain/revise_use_LINCS_HQ_data_pretrain_sample_id.csv')['x']
df_res = df_data[df_data['sig_id'].isin(use_HQ_sample_id)]
cellline_map = pd.read_csv('/egr/research-aidd/menghan1/AnchorDrug/data/CellLineEncode/use_training_cell_line_expression_features_128_encoded_20240111.csv')
cell_list = cellline_map['Unnamed: 0'].to_list()
df_res = df_res[df_res['cell_iname'].isin(cell_list)]

print(len(df_res))
pool_drug = set(df_res.SMILES.to_list())

for cell in ['MCF7', 'PC3', 'A549']:
        use_HQ_sample_id = pd.read_csv(f'/egr/research-aidd/menghan1/AnchorDrug/HQ_LINCS_retrain/revise_use_LINCS_HQ_data_target_cellline_{cell}_sample_id.csv')['x']
        # tmp = pd.read_csv('/egr/research-aidd/menghan1/AnchorDrug/HQ_LINCS_retrain/GPS_predictable_genes.csv')
        # genelist = tmp.x.to_list()
        
        
        df_target = df_data[df_data['sig_id'].isin(use_HQ_sample_id)]
        # median = df_target[['SMILES']+genelist].groupby(by='SMILES').median().reset_index()
        # median['cellline'] = cell
        # df_target = median
        df_target = df_target.rename(columns={'SMILES': 'smiles'})
        # for g in genelist:
        #     df_target[g] = df_target[g].apply(lambda x: (x > 1.5) * 1 + (x >= -1.5) * 1)

        Drugs = df_target['smiles'].to_list()
        tmp = pd.read_csv(f'/egr/research-aidd/menghan1/AnchorDrug/HQ_LINCS_retrain/{cell}_internal_val2_data.csv')
        valDrugs = tmp.SMILES.to_list()
        trainDrugs = set(Drugs)-set(valDrugs)
        print(len(Drugs), len(list(trainDrugs)), len(valDrugs))
        pool_drug = pool_drug.intersection(trainDrugs)
        print(len(list(pool_drug)))

        # df_train = df_target[df_target['smiles'].isin(trainDrugs)]
        # df_val = df_target[df_target['smiles'].isin(valDrugs)]


7258
2084 1203 134
810
2164 1272 141
497
1562 842 94
279


In [22]:
import pickle
with open('HQ_pool_drug.pkl', 'wb') as f:
    pickle.dump(list(pool_drug), f)

In [12]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import torch
import random
import os, pickle
from torchvision import datasets
import torch.utils.data as data
from sklearn.preprocessing import Normalizer
from sklearn.metrics import f1_score
from PIL import Image
from rdkit import Chem
from rdkit.Chem import AllChem

In [24]:
cell_list = ['MCF7']

In [38]:
        tmp = pd.read_csv('/egr/research-aidd/menghan1/AnchorDrug/HQ_LINCS_retrain/GPS_predictable_genes.csv')
        genelist = tmp.x.to_list()
        df_data = pd.read_csv('/egr/research-aidd/menghan1/AnchorDrug/data/level5_beta_trt_cp_24h_10uM.csv')

        for cell in cell_list:
            use_HQ_sample_id = pd.read_csv(f'/egr/research-aidd/menghan1/AnchorDrug/HQ_LINCS_retrain/revise_use_LINCS_HQ_data_target_cellline_{cell}_sample_id.csv')['x']
            df_target = df_data[df_data['sig_id'].isin(use_HQ_sample_id)]
            median = df_target[['SMILES']+genelist].groupby(by='SMILES').median()
            df_target = median[genelist]
            # df_target = df_target.rename(columns={'SMILES': 'smiles'})

            tmp = pd.read_csv(f'/egr/research-aidd/menghan1/AnchorDrug/HQ_LINCS_retrain/{cell}_internal_val2_data.csv')
            valDrugs = tmp.SMILES.to_list()
            with open('HQ_pool_drug.pkl', 'rb') as f:
                trainDrugs = pickle.load(f)
            # print(trainDrugs)

            df_train = df_target.loc[trainDrugs] #[df_target['smiles'].isin(trainDrugs)]
            # df_val = df_target[df_target['smiles'].isin(valDrugs)]

In [39]:
df_train

,LRP10,GABPB1,BLVRA,EGFR,FASTKD5,CNPY3,ELOVL6,ADGRG1,CCDC85B,CSNK2A2,...,NUP85,NPEPL1,TSC22D3,SPDEF,HSPA8,RFC2,ZNF318,DUSP6,ABCC5,SLC11A2
SMILES,,,,,,,,,,,,,,,,,,,,,
Nc1nc(Nc2ccc(S(N)(=O)=O)cc2)nn1C(=O)c1c(F)cccc1F,0.844790,-3.141880,-0.642785,0.130704,3.272202,3.447683,-7.026648,-0.282126,-0.302250,2.031959,...,-3.260277,0.040834,-1.407270,-4.356356,-7.639128,0.450629,-1.723505,1.057062,-1.309854,-1.907205
CCOc1ccccc1-n1c(C(C)N2CCN(C(=O)COc3ccc(Cl)cc3)CC2)nc2ccccc2c1=O,0.434109,-0.849405,1.453965,-0.184410,-0.526120,-0.555975,0.408255,1.409902,0.942473,-0.483885,...,-0.769631,0.047355,3.936394,0.356086,-1.499628,-0.136801,0.131799,-0.016127,3.001665,0.723271
COc1cc2c(cc1OC)[C@@H]1C(=O)c3ccc4c(c3O[C@@H]1CO2)C=CC(C)(C)O4,1.611638,0.049888,1.133430,0.566027,-0.096138,-0.490370,0.585708,0.938861,-1.441802,-0.419907,...,-1.884866,1.313013,5.951030,-0.673309,-1.678263,1.863695,-0.308549,0.681054,-0.060274,0.056928
COc1ccccc1NC(=O)N1CCCCN2[C@H](CO)[C@H](c3ccc(-c4cccc(C#N)c4)cc3)[C@H]2C1,1.010365,-1.238943,1.382840,0.619850,-0.100404,0.092938,4.171548,1.862624,-0.835273,-0.112110,...,0.004970,0.830357,1.514352,-0.286706,-1.082978,-0.541239,-0.610373,-0.591725,0.175310,1.119700
CC(C=CC=C(C)/C=C/C1=C(C)CCCC1(C)C)=CC(=O)O,-0.698101,-0.478742,-1.445821,-0.225239,-0.161361,0.578863,0.338546,-1.123500,-0.328465,-0.601196,...,0.511967,0.335509,3.918963,-2.866140,-0.726306,0.221207,0.287722,0.404384,-0.478100,0.368444
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
NC(=O)[C@@H](CCC(F)(F)F)N(Cc1ccc(-c2ncon2)cc1F)S(=O)(=O)c1ccc(Cl)cc1,-0.161300,-0.615540,1.343446,-1.101770,0.216882,3.040572,1.050093,0.813818,0.276921,-0.447311,...,0.588531,-0.759418,0.894103,-0.897958,-1.028421,0.687296,0.908807,-0.709292,0.167587,0.187313
Cc1cc(Nc2cc(CN3CCOCC3)c3nc(C)c(Cc4ccc(Cl)cc4F)n3n2)[nH]n1,-0.198444,-1.151405,1.346697,-0.067900,-1.200327,-0.344295,1.159928,-0.958113,-1.247008,0.741967,...,-2.585961,0.512763,0.125247,-2.036066,-0.454839,0.870651,-0.456121,0.547845,0.744508,2.489504
C[C@]12CC[C@H]3[C@@H](CC(=O)[C@H]4CC(=NOCCN)CC[C@@]43C)[C@@H]1CCC2=O,0.169609,0.195080,-4.566499,-0.866543,-0.199839,0.491439,-1.626103,2.912616,-0.835852,1.180923,...,-1.009045,-1.432807,1.535035,0.526571,0.646012,-0.826752,-1.997699,0.179970,0.529140,-0.092917


In [40]:
sum(genelist==df_train.T.index)

307

In [34]:
genelist

['LRP10',
 'GABPB1',
 'BLVRA',
 'EGFR',
 'FASTKD5',
 'CNPY3',
 'ELOVL6',
 'ADGRG1',
 'CCDC85B',
 'CSNK2A2',
 'ALDOC',
 'TRIB3',
 'BAMBI',
 'TCERG1',
 'S100A4',
 'RXYLT1',
 'LBR',
 'PSME1',
 'PPARD',
 'AKT1',
 'PDHX',
 'CLIC4',
 'PCBD1',
 'LIG1',
 'MYC',
 'MALT1',
 'SYNGR3',
 'KDM5B',
 'ECH1',
 'NSDHL',
 'NFKBIB',
 'ATP1B1',
 'RAP1GAP',
 'B4GAT1',
 'IGFBP3',
 'TSTA3',
 'KIF2C',
 'CLSTN1',
 'TXNRD1',
 'IL4R',
 'CALU',
 'PDGFA',
 'CHAC1',
 'STK25',
 'NOLC1',
 'HMGCS1',
 'CBR3',
 'TENT4A',
 'MYL9',
 'TRAK2',
 'PARP2',
 'PCNA',
 'TUBB6',
 'PRR7',
 'SCP2',
 'NCOA3',
 'GATA3',
 'OXSR1',
 'PRKCD',
 'KEAP1',
 'NMT1',
 'SLC25A4',
 'FAM57A',
 'HDAC2',
 'PAK6',
 'PIK3C2B',
 'P4HA2',
 'TCEA2',
 'NFIL3',
 'ICAM3',
 'CHIC2',
 'TSKU',
 'FOS',
 'TSPAN6',
 'RNMT',
 'FOXJ3',
 'HADH',
 'PYGL',
 'TMEM109',
 'LOXL1',
 'DNTTIP2',
 'PKIG',
 'ELAC2',
 'TMED10',
 'OXA1L',
 'ALDOA',
 'CTSL',
 'EBP',
 'DDIT4',
 'SPAG7',
 'CEBPA',
 'JADE2',
 'KDM3A',
 'WASHC5',
 'MBNL2',
 'SMARCD2',
 'MRPL12',
 'COASY',
 'PRSS23',

In [41]:
smiles_id = df_train.index
genelist = df_train.columns.to_list()
label_map = df_train.to_numpy()
data_smile = []
data_gene = []
data_label = []
for s in smiles_id.to_list():
    data_smile.extend([s]*len(genelist))
    data_gene.extend(genelist)
    data_label.extend(label_map[np.where(smiles_id == s)[0][0]])

In [48]:
        labels = torch.from_numpy(np.asarray(data_label))

In [49]:
labels.shape

torch.Size([85653])

In [45]:
np.where(smiles_id == s)

(array([278]),)

In [46]:
print(len(data_smile))
data_smile

85653


['Nc1nc(Nc2ccc(S(N)(=O)=O)cc2)nn1C(=O)c1c(F)cccc1F',
 'Nc1nc(Nc2ccc(S(N)(=O)=O)cc2)nn1C(=O)c1c(F)cccc1F',
 'Nc1nc(Nc2ccc(S(N)(=O)=O)cc2)nn1C(=O)c1c(F)cccc1F',
 'Nc1nc(Nc2ccc(S(N)(=O)=O)cc2)nn1C(=O)c1c(F)cccc1F',
 'Nc1nc(Nc2ccc(S(N)(=O)=O)cc2)nn1C(=O)c1c(F)cccc1F',
 'Nc1nc(Nc2ccc(S(N)(=O)=O)cc2)nn1C(=O)c1c(F)cccc1F',
 'Nc1nc(Nc2ccc(S(N)(=O)=O)cc2)nn1C(=O)c1c(F)cccc1F',
 'Nc1nc(Nc2ccc(S(N)(=O)=O)cc2)nn1C(=O)c1c(F)cccc1F',
 'Nc1nc(Nc2ccc(S(N)(=O)=O)cc2)nn1C(=O)c1c(F)cccc1F',
 'Nc1nc(Nc2ccc(S(N)(=O)=O)cc2)nn1C(=O)c1c(F)cccc1F',
 'Nc1nc(Nc2ccc(S(N)(=O)=O)cc2)nn1C(=O)c1c(F)cccc1F',
 'Nc1nc(Nc2ccc(S(N)(=O)=O)cc2)nn1C(=O)c1c(F)cccc1F',
 'Nc1nc(Nc2ccc(S(N)(=O)=O)cc2)nn1C(=O)c1c(F)cccc1F',
 'Nc1nc(Nc2ccc(S(N)(=O)=O)cc2)nn1C(=O)c1c(F)cccc1F',
 'Nc1nc(Nc2ccc(S(N)(=O)=O)cc2)nn1C(=O)c1c(F)cccc1F',
 'Nc1nc(Nc2ccc(S(N)(=O)=O)cc2)nn1C(=O)c1c(F)cccc1F',
 'Nc1nc(Nc2ccc(S(N)(=O)=O)cc2)nn1C(=O)c1c(F)cccc1F',
 'Nc1nc(Nc2ccc(S(N)(=O)=O)cc2)nn1C(=O)c1c(F)cccc1F',
 'Nc1nc(Nc2ccc(S(N)(=O)=O)cc2)nn1C(=O)c1c(F)cc

In [47]:
print(len(data_gene))
data_gene

85653


['LRP10',
 'GABPB1',
 'BLVRA',
 'EGFR',
 'FASTKD5',
 'CNPY3',
 'ELOVL6',
 'ADGRG1',
 'CCDC85B',
 'CSNK2A2',
 'ALDOC',
 'TRIB3',
 'BAMBI',
 'TCERG1',
 'S100A4',
 'RXYLT1',
 'LBR',
 'PSME1',
 'PPARD',
 'AKT1',
 'PDHX',
 'CLIC4',
 'PCBD1',
 'LIG1',
 'MYC',
 'MALT1',
 'SYNGR3',
 'KDM5B',
 'ECH1',
 'NSDHL',
 'NFKBIB',
 'ATP1B1',
 'RAP1GAP',
 'B4GAT1',
 'IGFBP3',
 'TSTA3',
 'KIF2C',
 'CLSTN1',
 'TXNRD1',
 'IL4R',
 'CALU',
 'PDGFA',
 'CHAC1',
 'STK25',
 'NOLC1',
 'HMGCS1',
 'CBR3',
 'TENT4A',
 'MYL9',
 'TRAK2',
 'PARP2',
 'PCNA',
 'TUBB6',
 'PRR7',
 'SCP2',
 'NCOA3',
 'GATA3',
 'OXSR1',
 'PRKCD',
 'KEAP1',
 'NMT1',
 'SLC25A4',
 'FAM57A',
 'HDAC2',
 'PAK6',
 'PIK3C2B',
 'P4HA2',
 'TCEA2',
 'NFIL3',
 'ICAM3',
 'CHIC2',
 'TSKU',
 'FOS',
 'TSPAN6',
 'RNMT',
 'FOXJ3',
 'HADH',
 'PYGL',
 'TMEM109',
 'LOXL1',
 'DNTTIP2',
 'PKIG',
 'ELAC2',
 'TMED10',
 'OXA1L',
 'ALDOA',
 'CTSL',
 'EBP',
 'DDIT4',
 'SPAG7',
 'CEBPA',
 'JADE2',
 'KDM3A',
 'WASHC5',
 'MBNL2',
 'SMARCD2',
 'MRPL12',
 'COASY',
 'PRSS23',

In [44]:
print(len(data_label))

85653


In [50]:
tmp_idxs = np.arange(10)

In [57]:
dataidx = []
for i in tmp_idxs:
    print([i*10+n for n in range(10)])
    dataidx.extend([i*10+n for n in range(10)])
print(dataidx)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
[10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
[20, 21, 22, 23, 24, 25, 26, 27, 28, 29]
[30, 31, 32, 33, 34, 35, 36, 37, 38, 39]
[40, 41, 42, 43, 44, 45, 46, 47, 48, 49]
[50, 51, 52, 53, 54, 55, 56, 57, 58, 59]
[60, 61, 62, 63, 64, 65, 66, 67, 68, 69]
[70, 71, 72, 73, 74, 75, 76, 77, 78, 79]
[80, 81, 82, 83, 84, 85, 86, 87, 88, 89]
[90, 91, 92, 93, 94, 95, 96, 97, 98, 99]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99]
